In [1]:
import sympy as sp
import numpy as np
from sympy import sqrt, Symbol, mathematica_code
from sympy import nsimplify as simp
from sympy.matrices import Matrix
from sympy.physics.quantum import TensorProduct, Dagger
from itertools import combinations
import qutip as qp
from multiprocessing.pool import Pool
from wolframclient.evaluation import WolframLanguageSession
from wolframclient.language import wlexpr
import datetime
import time
from sympy.parsing.mathematica import parse_mathematica

In [2]:
class Qubit:
    qubit_total = 0
    
    @classmethod
    def count_qubit(cls):
        cls.qubit_total += 1

    def __init__(self, a, b):
        self.qubit_num = self.qubit_total
        self.count_qubit()
        
        self.a = sp.nsimplify(a)
        self.b = sp.nsimplify(b)
            
        self.state = Matrix([self.a, self.b])
        
    @property
    def dag(self):
        return Dagger(self.state)
    
#     def __mul__(self, other):
#         return Qubitself.state * other
    
    def __repr__(self):
        return repr(self.state)
        
def matrix_to_braket(matrix, qubit_num=None):
    cols = matrix.cols
    rows = matrix.rows
    def binary(x):
        if qubit_num is None:
            return x
        else:
            num = f"{x:0>b}"
            return (qubit_num-len(num))*'0'+num
        
    
    col_symbols = [sp.Symbol(f"|{binary(i)}>", commutative=False) for i in range(cols)]
    row_symbols = [sp.Symbol(f"<{binary(i)}|", commutative=False) for i in range(rows)]
    
    expression = 0
    
    for i, element in enumerate(matrix):
        row = i // cols
        col = i % cols
        
        expression += element*col_symbols[col]*row_symbols[row]
        
    return expression

In [3]:
def search_all_combinations(group):
    """Find the summation of all possible combinations of groups"""
    max_len = int(len(group)/2)
    # Add empty set edge case
    sub_groups = {frozenset(): sp.Integer(0), frozenset(group.keys()): sp.Integer(1)}

    
    for i in range(max_len):
        combs = combinations(group, i+1)
        for selection in combs:
            duplicate_group = dict(group)
            for prob in selection:
                duplicate_group.pop(prob)
            list1 = [group.get(key) for key in selection]
            list2 = list(duplicate_group.values())
            
            sub_groups[frozenset(selection)] = sum(list1)
            sub_groups[frozenset(duplicate_group.keys())] = sum(list2)
    
    return sub_groups

def basic_measure(calc_prob, a, b):
    prob = calc_prob - a*sp.conjugate(a)
    prob = prob.subs(b, sqrt(1-a*sp.conjugate(a))).evalf()
    measure = sqrt(sp.Integral(prob**2, (a, 0, 1)))
        
    return measure

def measure_all_probabilities(group_probabilities, a, b, measure_func=basic_measure):
    total_measures = len(group_probabilities)
    print(f"Calculating {total_measures} measures...")
    
    measures = {}
    start = time.time()
    for i, (key, item) in enumerate(group_probabilities.items()):
        if i%100 == 0 and i != 0:
                duration = time.time() - start
                calc_per_sec = i/duration
                time_remaining = (total_measures - i)/(calc_per_sec)
                hours = time_remaining//3600
                minutes = (time_remaining%3600)//60
                print(f"Calculated {i} measures, {calc_per_sec} it/s, {hours}H{minutes}M remaining")

        measure = sp.simplify(measure_func(item, a, b)).evalf(chop=True)
        measures[key] = measure 
        
    return measures

def lowest_measure(group_measures):
    lowest_val = np.inf
    for key, item in group_measures.items():
        if item <= lowest_val:
            if item == lowest_val:
                lowest_key.add(key)
            else:
                lowest_key = {key}
            lowest_val = item
            
    return lowest_key, lowest_val

def make_tensor(povm):
    tensor = {}
    for i, p1 in enumerate(povm):
        for j, p2 in enumerate(povm):
            tensor[f"({i},{j})"] = TensorProduct(p1, p2)
            
    return tensor
            
def calc_probabilities(tensor, a, b):
    psi = Matrix([0]*tensor.cols)
    psi[0] = a
    psi[-1] = b

    return (Dagger(psi)*tensor*psi)[0]

In [4]:
q0 = Qubit(1, 0)
q1 = Qubit(0, 1)

phi1 = Qubit(1/2, sqrt(3)/2)
phi2 = Qubit(1/2, -sqrt(3)/2)

In [5]:
M0 = simp(2/3) * q0.state*q0.dag
M1 = simp(2/3) * phi1.state*phi1.dag
M2 = simp(2/3) * phi2.state*phi2.dag

trine = [M0, M1, M2]

In [13]:
trine_tensor = make_tensor(trine)

In [14]:
trine_tensor

{'P(0,0)': Matrix([
 [4/9, 0, 0, 0],
 [  0, 0, 0, 0],
 [  0, 0, 0, 0],
 [  0, 0, 0, 0]]),
 'P(0,1)': Matrix([
 [      1/9, sqrt(3)/9, 0, 0],
 [sqrt(3)/9,       1/3, 0, 0],
 [        0,         0, 0, 0],
 [        0,         0, 0, 0]]),
 'P(0,2)': Matrix([
 [       1/9, -sqrt(3)/9, 0, 0],
 [-sqrt(3)/9,        1/3, 0, 0],
 [         0,          0, 0, 0],
 [         0,          0, 0, 0]]),
 'P(1,0)': Matrix([
 [      1/9, 0, sqrt(3)/9, 0],
 [        0, 0,         0, 0],
 [sqrt(3)/9, 0,       1/3, 0],
 [        0, 0,         0, 0]]),
 'P(1,1)': Matrix([
 [      1/36, sqrt(3)/36, sqrt(3)/36,       1/12],
 [sqrt(3)/36,       1/12,       1/12, sqrt(3)/12],
 [sqrt(3)/36,       1/12,       1/12, sqrt(3)/12],
 [      1/12, sqrt(3)/12, sqrt(3)/12,        1/4]]),
 'P(1,2)': Matrix([
 [       1/36, -sqrt(3)/36,  sqrt(3)/36,       -1/12],
 [-sqrt(3)/36,        1/12,       -1/12,  sqrt(3)/12],
 [ sqrt(3)/36,       -1/12,        1/12, -sqrt(3)/12],
 [      -1/12,  sqrt(3)/12, -sqrt(3)/12,         1/4]

In [15]:
matrix_to_braket(trine_tensor["P(2,2)"], 2)

|00>*<00|/36 - sqrt(3)*|00>*<01|/36 - sqrt(3)*|00>*<10|/36 + |00>*<11|/12 - sqrt(3)*|01>*<00|/36 + |01>*<01|/12 + |01>*<10|/12 - sqrt(3)*|01>*<11|/12 - sqrt(3)*|10>*<00|/36 + |10>*<01|/12 + |10>*<10|/12 - sqrt(3)*|10>*<11|/12 + |11>*<00|/12 - sqrt(3)*|11>*<01|/12 - sqrt(3)*|11>*<10|/12 + |11>*<11|/4

In [16]:
a = sp.Symbol("a", real=True)
b = sp.Symbol("b", real=True)

probabilities = {}
for key, tri in trine_tensor.items():
    probabilities[key] = calc_probabilities(tri, a, b)

In [17]:
[probabilities["P(0,0)"], probabilities["P(0,1)"]]

[4*a**2/9, a**2/9]

In [22]:
groups = search_all_combinations(probabilities)

In [80]:
all_measures = measure_all_probabilities(groups, a, b)

Calculating 512 measures...
Calculated 10 measures...
Calculated 20 measures...
Calculated 30 measures...
Calculated 40 measures...
Calculated 50 measures...
Calculated 60 measures...
Calculated 70 measures...
Calculated 80 measures...
Calculated 90 measures...
Calculated 100 measures...
Calculated 110 measures...
Calculated 120 measures...
Calculated 130 measures...
Calculated 140 measures...
Calculated 150 measures...
Calculated 160 measures...
Calculated 170 measures...
Calculated 180 measures...
Calculated 190 measures...
Calculated 200 measures...
Calculated 210 measures...
Calculated 220 measures...
Calculated 230 measures...
Calculated 240 measures...
Calculated 250 measures...
Calculated 260 measures...
Calculated 270 measures...
Calculated 280 measures...
Calculated 290 measures...
Calculated 300 measures...
Calculated 310 measures...
Calculated 320 measures...
Calculated 330 measures...
Calculated 340 measures...
Calculated 350 measures...
Calculated 360 measures...
Calculate

In [87]:
group, measure = lowest_measure(all_measures)
group

{frozenset({'P(0,0)', 'P(0,1)', 'P(0,2)', 'P(1,0)', 'P(2,0)'})}

Tetra-measurement

In [50]:
phi0 = Qubit(1, 0)
phi1 = Qubit(1/sqrt(3), sqrt(2/3))
phi2 = Qubit(1/sqrt(3), sp.E**(sp.I*2*sp.pi/3)*sqrt(2/3))
phi3 = Qubit(1/sqrt(3), sp.E**(-sp.I*2*sp.pi/3)*sqrt(2/3))


M0 = simp(1/2)*phi0.state*phi0.dag
M1 = simp(1/2)*phi1.state*phi1.dag
M2 = simp(1/2)*phi2.state*phi2.dag
M3 = simp(1/2)*phi3.state*phi3.dag

In [51]:
sp.simplify(M0+M1+M2+M3)

Matrix([
[1, 0],
[0, 1]])

In [10]:
%matplotlib notebook

b3d = qp.Bloch3d()
b3d.make_sphere()

x1 = (phi0.a.evalf()*qp.basis(2, 0) + complex(phi0.b.evalf())*qp.basis(2, 1)).unit()
x2 = (phi1.a.evalf()*qp.basis(2, 0) + complex(phi1.b.evalf())*qp.basis(2, 1)).unit()
x3 = (phi2.a.evalf()*qp.basis(2, 0) + complex(phi2.b.evalf())*qp.basis(2, 1)).unit()
x4 = (phi3.a.evalf()*qp.basis(2, 0) + complex(phi3.b.evalf())*qp.basis(2, 1)).unit()

b3d.add_states([x1, x2, x3, x4])
b3d.show()

In [66]:
sp.simplify(matrix_to_braket(M2))

(2*|0>*<0| - sqrt(2)*|0>*<1| + sqrt(6)*I*|0>*<1| - sqrt(2)*|1>*<0| - sqrt(6)*I*|1>*<0| + 4*|1>*<1|)/12

In [52]:
tetra = [M0, M1, M2, M3]

tetra_tensor = {}
for i in range(4):
    for j in range(4):
        tetra_tensor[f"P({i},{j})"] = TensorProduct(tetra[i], tetra[j])

In [165]:
a = sp.Symbol("a", real=True)
b = sp.Symbol("b", real=True)
psi = Matrix([a, 0, 0, b])

probabilities = {}
for key, tri in trine_tensor.items():
    probabilities[key] = (Dagger(psi)*tri*psi)[0]

In [168]:
groups = search_all_combinations(probabilities)

In [157]:
groups

[{('P(0,0)',): a*conjugate(a)/4,
  ('P(0,1)',
   'P(0,2)',
   'P(1,0)',
   'P(1,1)',
   'P(1,2)',
   'P(2,0)',
   'P(2,1)',
   'P(2,2)',
   'P(0,3)',
   'P(1,3)',
   'P(2,3)',
   'P(3,0)',
   'P(3,1)',
   'P(3,2)',
   'P(3,3)'): a*(conjugate(a)/36 + (-sqrt(6)/12 - sqrt(2)*I/4)**2*conjugate(b)/3) + a*(conjugate(a)/36 + (-sqrt(6)/12 + sqrt(2)*I/4)**2*conjugate(b)/3) + 2*a*(conjugate(a)/36 + sqrt(6)*(-sqrt(6)/12 - sqrt(2)*I/4)*conjugate(b)/18) + 2*a*(conjugate(a)/36 + sqrt(6)*(-sqrt(6)/12 + sqrt(2)*I/4)*conjugate(b)/18) + 2*a*(conjugate(a)/36 + (-sqrt(6)/12 - sqrt(2)*I/4)*(-sqrt(6)/12 + sqrt(2)*I/4)*conjugate(b)/3) + a*(conjugate(a)/36 + conjugate(b)/18) + a*conjugate(a)/2 + b*((-sqrt(6)/6 - sqrt(2)*I/2)**2*conjugate(a)/12 + (-sqrt(6)/6 - sqrt(2)*I/2)**2*(-sqrt(6)/12 + sqrt(2)*I/4)**2*conjugate(b)) + b*((-sqrt(6)/6 + sqrt(2)*I/2)**2*conjugate(a)/12 + (-sqrt(6)/6 + sqrt(2)*I/2)**2*(-sqrt(6)/12 - sqrt(2)*I/4)**2*conjugate(b)) + 2*b*(sqrt(6)*(-sqrt(6)/6 - sqrt(2)*I/2)*conjugate(a)/36 + (-sqr

In [224]:
all_measures = measure_all_probabilities(groups, a, b)

Calculating 78404 measures...
Calculated 20 measures...
Calculated 40 measures...
Calculated 60 measures...
Calculated 80 measures...
Calculated 100 measures...
Calculated 120 measures...
Calculated 140 measures...
Calculated 160 measures...
Calculated 180 measures...
Calculated 200 measures...
Calculated 220 measures...
Calculated 240 measures...
Calculated 260 measures...
Calculated 280 measures...
Calculated 300 measures...
Calculated 320 measures...
Calculated 340 measures...
Calculated 360 measures...
Calculated 380 measures...
Calculated 400 measures...
Calculated 420 measures...
Calculated 440 measures...
Calculated 460 measures...
Calculated 480 measures...
Calculated 500 measures...
Calculated 520 measures...
Calculated 540 measures...
Calculated 560 measures...
Calculated 580 measures...
Calculated 600 measures...
Calculated 620 measures...
Calculated 640 measures...
Calculated 660 measures...
Calculated 680 measures...
Calculated 700 measures...
Calculated 720 measures...
Ca

Calculated 5900 measures...
Calculated 5920 measures...
Calculated 5940 measures...
Calculated 5960 measures...
Calculated 5980 measures...
Calculated 6000 measures...
Calculated 6020 measures...
Calculated 6040 measures...
Calculated 6060 measures...
Calculated 6080 measures...
Calculated 6100 measures...
Calculated 6120 measures...
Calculated 6140 measures...
Calculated 6160 measures...
Calculated 6180 measures...
Calculated 6200 measures...
Calculated 6220 measures...
Calculated 6240 measures...
Calculated 6260 measures...
Calculated 6280 measures...
Calculated 6300 measures...
Calculated 6320 measures...
Calculated 6340 measures...
Calculated 6360 measures...
Calculated 6380 measures...
Calculated 6400 measures...
Calculated 6420 measures...
Calculated 6440 measures...
Calculated 6460 measures...
Calculated 6480 measures...
Calculated 6500 measures...
Calculated 6520 measures...
Calculated 6540 measures...
Calculated 6560 measures...
Calculated 6580 measures...
Calculated 6600 meas

Calculated 11700 measures...
Calculated 11720 measures...
Calculated 11740 measures...
Calculated 11760 measures...
Calculated 11780 measures...
Calculated 11800 measures...
Calculated 11820 measures...
Calculated 11840 measures...
Calculated 11860 measures...
Calculated 11880 measures...
Calculated 11900 measures...
Calculated 11920 measures...
Calculated 11940 measures...
Calculated 11960 measures...
Calculated 11980 measures...
Calculated 12000 measures...
Calculated 12020 measures...
Calculated 12040 measures...
Calculated 12060 measures...
Calculated 12080 measures...
Calculated 12100 measures...
Calculated 12120 measures...
Calculated 12140 measures...
Calculated 12160 measures...
Calculated 12180 measures...
Calculated 12200 measures...
Calculated 12220 measures...
Calculated 12240 measures...
Calculated 12260 measures...
Calculated 12280 measures...
Calculated 12300 measures...
Calculated 12320 measures...
Calculated 12340 measures...
Calculated 12360 measures...
Calculated 123

Calculated 17360 measures...
Calculated 17380 measures...
Calculated 17400 measures...
Calculated 17420 measures...
Calculated 17440 measures...
Calculated 17460 measures...
Calculated 17480 measures...
Calculated 17500 measures...
Calculated 17520 measures...
Calculated 17540 measures...
Calculated 17560 measures...
Calculated 17580 measures...
Calculated 17600 measures...
Calculated 17620 measures...
Calculated 17640 measures...
Calculated 17660 measures...
Calculated 17680 measures...
Calculated 17700 measures...
Calculated 17720 measures...
Calculated 17740 measures...
Calculated 17760 measures...
Calculated 17780 measures...
Calculated 17800 measures...
Calculated 17820 measures...
Calculated 17840 measures...
Calculated 17860 measures...
Calculated 17880 measures...
Calculated 17900 measures...
Calculated 17920 measures...
Calculated 17940 measures...
Calculated 17960 measures...
Calculated 17980 measures...
Calculated 18000 measures...
Calculated 18020 measures...
Calculated 180

Calculated 23020 measures...
Calculated 23040 measures...
Calculated 23060 measures...
Calculated 23080 measures...
Calculated 23100 measures...
Calculated 23120 measures...
Calculated 23140 measures...
Calculated 23160 measures...
Calculated 23180 measures...
Calculated 23200 measures...
Calculated 23220 measures...
Calculated 23240 measures...
Calculated 23260 measures...
Calculated 23280 measures...
Calculated 23300 measures...
Calculated 23320 measures...
Calculated 23340 measures...
Calculated 23360 measures...
Calculated 23380 measures...
Calculated 23400 measures...
Calculated 23420 measures...
Calculated 23440 measures...
Calculated 23460 measures...
Calculated 23480 measures...
Calculated 23500 measures...
Calculated 23520 measures...
Calculated 23540 measures...
Calculated 23560 measures...
Calculated 23580 measures...
Calculated 23600 measures...
Calculated 23620 measures...
Calculated 23640 measures...
Calculated 23660 measures...
Calculated 23680 measures...
Calculated 237

...

Calculated 51320 measures...
Calculated 51340 measures...
Calculated 51360 measures...
Calculated 51380 measures...
Calculated 51400 measures...
Calculated 51420 measures...
Calculated 51440 measures...
Calculated 51460 measures...
Calculated 51480 measures...
Calculated 51500 measures...
Calculated 51520 measures...
Calculated 51540 measures...
Calculated 51560 measures...
Calculated 51580 measures...
Calculated 51600 measures...
Calculated 51620 measures...
Calculated 51640 measures...
Calculated 51660 measures...
Calculated 51680 measures...
Calculated 51700 measures...
Calculated 51720 measures...
Calculated 51740 measures...
Calculated 51760 measures...
Calculated 51780 measures...
Calculated 51800 measures...
Calculated 51820 measures...
Calculated 51840 measures...
Calculated 51860 measures...
Calculated 51880 measures...
Calculated 51900 measures...
Calculated 51920 measures...
Calculated 51940 measures...
Calculated 51960 measures...
Calculated 51980 measures...
Calculated 520

Calculated 56980 measures...
Calculated 57000 measures...
Calculated 57020 measures...
Calculated 57040 measures...
Calculated 57060 measures...
Calculated 57080 measures...
Calculated 57100 measures...
Calculated 57120 measures...
Calculated 57140 measures...
Calculated 57160 measures...
Calculated 57180 measures...
Calculated 57200 measures...
Calculated 57220 measures...
Calculated 57240 measures...
Calculated 57260 measures...
Calculated 57280 measures...
Calculated 57300 measures...
Calculated 57320 measures...
Calculated 57340 measures...
Calculated 57360 measures...
Calculated 57380 measures...
Calculated 57400 measures...
Calculated 57420 measures...
Calculated 57440 measures...
Calculated 57460 measures...
Calculated 57480 measures...
Calculated 57500 measures...
Calculated 57520 measures...
Calculated 57540 measures...
Calculated 57560 measures...
Calculated 57580 measures...
Calculated 57600 measures...
Calculated 57620 measures...
Calculated 57640 measures...
Calculated 576

Calculated 62640 measures...
Calculated 62660 measures...
Calculated 62680 measures...
Calculated 62700 measures...
Calculated 62720 measures...
Calculated 62740 measures...
Calculated 62760 measures...
Calculated 62780 measures...
Calculated 62800 measures...
Calculated 62820 measures...
Calculated 62840 measures...
Calculated 62860 measures...
Calculated 62880 measures...
Calculated 62900 measures...
Calculated 62920 measures...
Calculated 62940 measures...
Calculated 62960 measures...
Calculated 62980 measures...
Calculated 63000 measures...
Calculated 63020 measures...
Calculated 63040 measures...
Calculated 63060 measures...
Calculated 63080 measures...
Calculated 63100 measures...
Calculated 63120 measures...
Calculated 63140 measures...
Calculated 63160 measures...
Calculated 63180 measures...
Calculated 63200 measures...
Calculated 63220 measures...
Calculated 63240 measures...
Calculated 63260 measures...
Calculated 63280 measures...
Calculated 63300 measures...
Calculated 633

Calculated 68300 measures...
Calculated 68320 measures...
Calculated 68340 measures...
Calculated 68360 measures...
Calculated 68380 measures...
Calculated 68400 measures...
Calculated 68420 measures...
Calculated 68440 measures...
Calculated 68460 measures...
Calculated 68480 measures...
Calculated 68500 measures...
Calculated 68520 measures...
Calculated 68540 measures...
Calculated 68560 measures...
Calculated 68580 measures...
Calculated 68600 measures...
Calculated 68620 measures...
Calculated 68640 measures...
Calculated 68660 measures...
Calculated 68680 measures...
Calculated 68700 measures...
Calculated 68720 measures...
Calculated 68740 measures...
Calculated 68760 measures...
Calculated 68780 measures...
Calculated 68800 measures...
Calculated 68820 measures...
Calculated 68840 measures...
Calculated 68860 measures...
Calculated 68880 measures...
Calculated 68900 measures...
Calculated 68920 measures...
Calculated 68940 measures...
Calculated 68960 measures...
Calculated 689

Calculated 73960 measures...
Calculated 73980 measures...
Calculated 74000 measures...
Calculated 74020 measures...
Calculated 74040 measures...
Calculated 74060 measures...
Calculated 74080 measures...
Calculated 74100 measures...
Calculated 74120 measures...
Calculated 74140 measures...
Calculated 74160 measures...
Calculated 74180 measures...
Calculated 74200 measures...
Calculated 74220 measures...
Calculated 74240 measures...
Calculated 74260 measures...
Calculated 74280 measures...
Calculated 74300 measures...
Calculated 74320 measures...
Calculated 74340 measures...
Calculated 74360 measures...
Calculated 74380 measures...
Calculated 74400 measures...
Calculated 74420 measures...
Calculated 74440 measures...
Calculated 74460 measures...
Calculated 74480 measures...
Calculated 74500 measures...
Calculated 74520 measures...
Calculated 74540 measures...
Calculated 74560 measures...
Calculated 74580 measures...
Calculated 74600 measures...
Calculated 74620 measures...
Calculated 746

In [225]:
group, position = lowest_measure(all_measures)
group

('P(0,0)',
 'P(0,1)',
 'P(0,2)',
 'P(1,0)',
 'P(1,1)',
 'P(2,0)',
 'P(0,3)',
 'P(3,0)')

Tetra measurement again but using the proper measure of error

In [64]:
def root_mean_square_measure(calc_prob, a, b):
    theta = sp.Symbol("θ", real=True)
    phi = sp.Symbol("𝜙", real=True)
    
    prob = calc_prob - a*sp.conjugate(a)
    prob = prob.subs({a: sp.cos(theta/2), b: sp.E**(sp.I*phi)*sp.sin(theta/2)})
    measure = sqrt(sp.Integral(sp.sin(theta)/(4*sp.pi)*prob**2, (theta, 0, sp.pi), (phi, 0, 2*sp.pi)))
    
    return measure

def measure_all_probabilities_mathematica(group_probabilities, a, b, measure_func=basic_measure):
    total_measures = len(group_probabilities)
    print(f"Calculating {total_measures} measures...")
    
    measures = {}
    start = time.time()
    with WolframLanguageSession('C:\\Program Files\\Wolfram Research\\Wolfram Engine\\13.2\\WolframKernel.exe') as session:
        for i, (key, item) in enumerate(group_probabilities.items()):
            if i%100 == 0 and i != 0:
                duration = time.time() - start
                calc_per_sec = i/duration
                time_remaining = (total_measures - i)/(calc_per_sec)
                hours = time_remaining//3600
                minutes = (time_remaining%3600)//60
                print(f"Calculated {i} measures, {calc_per_sec} it/s, {hours}H{minutes}M remaining")
                
                
            command = measure_func(item, a, b)
            command = 'N[ReleaseHold[' + mathematica_code(command) + ']]'
            measure = session.evaluate(wlexpr(command))
            
            measures[key] = measure
        
    return measures

Trine with mathemtica and Haar measure

In [12]:
q0 = Qubit(1, 0)
q1 = Qubit(0, 1)

phi1 = Qubit(1/2, sqrt(3)/2)
phi2 = Qubit(1/2, -sqrt(3)/2)

M0 = simp(2/3) * q0.state*q0.dag
M1 = simp(2/3) * phi1.state*phi1.dag
M2 = simp(2/3) * phi2.state*phi2.dag

trine = [M0, M1, M2]

trine_tensor = make_tensor(trine)

a = sp.Symbol("a")
b = sp.Symbol("b")

probabilities = {}
for key, tri in trine_tensor.items():
    probabilities[key] = calc_probabilities(tri, a, b)
    
groups = search_all_combinations(probabilities)

In [ ]:
all_measures = measure_all_probabilities_mathematica(groups, a, b, root_mean_square_measure)

In [8]:
group, measure = lowest_measure(all_measures)
group

NameError: name 'all_measures' is not defined

In [46]:
with open(f"two_trine_povm_measurements_{datetime.datetime.now().strftime('%Y_%m_%d-%H_%M_%S')}.txt", "w") as f:
    f.write(str(all_measures))

Tetra with mathemtica and Haar measure

In [25]:
phi0 = Qubit(1, 0)
phi1 = Qubit(1 / sqrt(3), sqrt(2 / 3))
phi2 = Qubit(1 / sqrt(3), sp.E ** (sp.I * 2 * sp.pi / 3) * sqrt(2 / 3))
phi3 = Qubit(1 / sqrt(3), sp.E ** (-sp.I * 2 * sp.pi / 3) * sqrt(2 / 3))

M0 = simp(1 / 2) * phi0.state * phi0.dag
M1 = simp(1 / 2) * phi1.state * phi1.dag
M2 = simp(1 / 2) * phi2.state * phi2.dag
M3 = simp(1 / 2) * phi3.state * phi3.dag

tetra = [M0, M1, M2, M3]

tetra_tensor = make_tensor(tetra)

a = sp.Symbol("a")
b = sp.Symbol("b")

probabilities = {}
for key, tet in tetra_tensor.items():
    probabilities[key] = calc_probabilities(tet, a, b)

groups = search_all_combinations(probabilities)

In [16]:
all_measures = measure_all_probabilities_mathematica(groups, a, b, root_mean_square_measure)

Calculating 65536 measures...
Calculated 100 measures, 1.0731838618017864 it/s, 16.0H56.0M remaining
Calculated 200 measures, 1.0422346095028352 it/s, 17.0H24.0M remaining
Calculated 300 measures, 1.037137239663266 it/s, 17.0H28.0M remaining
Calculated 400 measures, 1.0085447976585245 it/s, 17.0H56.0M remaining
Calculated 500 measures, 0.9639119186213656 it/s, 18.0H44.0M remaining
Calculated 600 measures, 0.9093301804318853 it/s, 19.0H50.0M remaining
Calculated 700 measures, 0.8976516327080628 it/s, 20.0H3.0M remaining
Calculated 800 measures, 0.8800337660679874 it/s, 20.0H26.0M remaining
Calculated 900 measures, 0.8845178245569829 it/s, 20.0H17.0M remaining
Calculated 1000 measures, 0.8876532020893899 it/s, 20.0H11.0M remaining
Calculated 1100 measures, 0.8779032063691251 it/s, 20.0H23.0M remaining
Calculated 1200 measures, 0.8494696533504629 it/s, 21.0H2.0M remaining
Calculated 1300 measures, 0.8362499151926646 it/s, 21.0H20.0M remaining
Calculated 1400 measures, 0.8379012583416814 i

Calculated 11500 measures, 0.5807016638642495 it/s, 25.0H50.0M remaining
Calculated 11600 measures, 0.5805519421695965 it/s, 25.0H48.0M remaining
Calculated 11700 measures, 0.5800482817880435 it/s, 25.0H46.0M remaining
Calculated 11800 measures, 0.5787402270671084 it/s, 25.0H47.0M remaining
Calculated 11900 measures, 0.5781111918174975 it/s, 25.0H46.0M remaining
Calculated 12000 measures, 0.577644730769985 it/s, 25.0H44.0M remaining
Calculated 12100 measures, 0.5774256831162317 it/s, 25.0H42.0M remaining
Calculated 12200 measures, 0.5774523338844116 it/s, 25.0H39.0M remaining
Calculated 12300 measures, 0.5767736548938774 it/s, 25.0H38.0M remaining
Calculated 12400 measures, 0.5762799023719912 it/s, 25.0H36.0M remaining
Calculated 12500 measures, 0.5753933081742324 it/s, 25.0H36.0M remaining
Calculated 12600 measures, 0.5747741372193645 it/s, 25.0H34.0M remaining
Calculated 12700 measures, 0.5740828831268858 it/s, 25.0H33.0M remaining
Calculated 12800 measures, 0.574440599679655 it/s, 2

Calculated 22800 measures, 0.5330699846610589 it/s, 22.0H16.0M remaining
Calculated 22900 measures, 0.5327291776815917 it/s, 22.0H13.0M remaining
Calculated 23000 measures, 0.5320560845461925 it/s, 22.0H12.0M remaining
Calculated 23100 measures, 0.5315507526666121 it/s, 22.0H10.0M remaining
Calculated 23200 measures, 0.5310594941928487 it/s, 22.0H8.0M remaining
Calculated 23300 measures, 0.5306663700975477 it/s, 22.0H6.0M remaining
Calculated 23400 measures, 0.53012148379517 it/s, 22.0H4.0M remaining
Calculated 23500 measures, 0.5295152964894012 it/s, 22.0H3.0M remaining
Calculated 23600 measures, 0.529151173713531 it/s, 22.0H0.0M remaining
Calculated 23700 measures, 0.5286660082763923 it/s, 21.0H58.0M remaining
Calculated 23800 measures, 0.528548149759304 it/s, 21.0H56.0M remaining
Calculated 23900 measures, 0.5286366700823897 it/s, 21.0H52.0M remaining
Calculated 24000 measures, 0.5287038302203231 it/s, 21.0H49.0M remaining
Calculated 24100 measures, 0.5289219634364174 it/s, 21.0H45.

Calculated 34100 measures, 0.5075747789881838 it/s, 17.0H12.0M remaining
Calculated 34200 measures, 0.5074791760560209 it/s, 17.0H9.0M remaining
Calculated 34300 measures, 0.5073648642508963 it/s, 17.0H6.0M remaining
Calculated 34400 measures, 0.5071052018574226 it/s, 17.0H3.0M remaining
Calculated 34500 measures, 0.5069167607690864 it/s, 17.0H0.0M remaining
Calculated 34600 measures, 0.5067844986020676 it/s, 16.0H57.0M remaining
Calculated 34700 measures, 0.5066923144135091 it/s, 16.0H54.0M remaining
Calculated 34800 measures, 0.5067398866665819 it/s, 16.0H50.0M remaining
Calculated 34900 measures, 0.5065480145161705 it/s, 16.0H47.0M remaining
Calculated 35000 measures, 0.5064020721155912 it/s, 16.0H44.0M remaining
Calculated 35100 measures, 0.5061997213308415 it/s, 16.0H42.0M remaining
Calculated 35200 measures, 0.5060474225586646 it/s, 16.0H39.0M remaining
Calculated 35300 measures, 0.5059075329935604 it/s, 16.0H36.0M remaining
Calculated 35400 measures, 0.505875608473724 it/s, 16.0

Calculated 45400 measures, 0.4974347164157152 it/s, 11.0H14.0M remaining
Calculated 45500 measures, 0.49732794005379166 it/s, 11.0H11.0M remaining
Calculated 45600 measures, 0.4971841583551103 it/s, 11.0H8.0M remaining
Calculated 45700 measures, 0.49701866707658693 it/s, 11.0H5.0M remaining
Calculated 45800 measures, 0.49697702950434564 it/s, 11.0H1.0M remaining
Calculated 45900 measures, 0.49706079229695893 it/s, 10.0H58.0M remaining
Calculated 46000 measures, 0.4971210586564888 it/s, 10.0H54.0M remaining
Calculated 46100 measures, 0.4970663604675608 it/s, 10.0H51.0M remaining
Calculated 46200 measures, 0.4969985643666099 it/s, 10.0H48.0M remaining
Calculated 46300 measures, 0.4969214492847048 it/s, 10.0H45.0M remaining
Calculated 46400 measures, 0.4970330610478206 it/s, 10.0H41.0M remaining
Calculated 46500 measures, 0.49706665089369195 it/s, 10.0H38.0M remaining
Calculated 46600 measures, 0.49688628461341927 it/s, 10.0H35.0M remaining
Calculated 46700 measures, 0.49684630396948803 i

Calculated 56800 measures, 0.49534729509660297 it/s, 4.0H53.0M remaining
Calculated 56900 measures, 0.49534311250683283 it/s, 4.0H50.0M remaining
Calculated 57000 measures, 0.49529750284208834 it/s, 4.0H47.0M remaining
Calculated 57100 measures, 0.4952131919084022 it/s, 4.0H43.0M remaining
Calculated 57200 measures, 0.4952990432690092 it/s, 4.0H40.0M remaining
Calculated 57300 measures, 0.4953244498640617 it/s, 4.0H37.0M remaining
Calculated 57400 measures, 0.49531561463809004 it/s, 4.0H33.0M remaining
Calculated 57500 measures, 0.49527204691231325 it/s, 4.0H30.0M remaining
Calculated 57600 measures, 0.4952979289039569 it/s, 4.0H27.0M remaining
Calculated 57700 measures, 0.4953787834465038 it/s, 4.0H23.0M remaining
Calculated 57800 measures, 0.4953726128140717 it/s, 4.0H20.0M remaining
Calculated 57900 measures, 0.4953119256408929 it/s, 4.0H16.0M remaining
Calculated 58000 measures, 0.49525174649531567 it/s, 4.0H13.0M remaining
Calculated 58100 measures, 0.4951736506885314 it/s, 4.0H10

In [107]:
with open(f"two_tetra_povm_measurements_{datetime.datetime.now().strftime('%Y_%m_%d-%H_%M_%S')}.txt", "w") as f:
    f.write(str(all_measures))

In [19]:
group, measure = lowest_measure(all_measures)
print(group, measure)

{frozenset({'P(1,0)', 'P(3,0)', 'P(0,2)', 'P(0,1)', 'P(0,0)', 'P(0,3)', 'P(3,2)', 'P(2,0)'}), frozenset({'P(1,0)', 'P(3,0)', 'P(0,2)', 'P(0,1)', 'P(0,0)', 'P(1,3)', 'P(0,3)', 'P(2,0)'}), frozenset({'P(1,0)', 'P(3,0)', 'P(0,2)', 'P(0,1)', 'P(0,0)', 'P(1,2)', 'P(0,3)', 'P(2,0)'}), frozenset({'P(1,0)', 'P(3,0)', 'P(0,2)', 'P(0,1)', 'P(0,0)', 'P(0,3)', 'P(1,1)', 'P(2,0)'}), frozenset({'P(1,0)', 'P(3,0)', 'P(0,2)', 'P(0,1)', 'P(0,0)', 'P(0,3)', 'P(2,2)', 'P(2,0)'}), frozenset({'P(1,0)', 'P(3,0)', 'P(0,2)', 'P(0,1)', 'P(0,0)', 'P(3,3)', 'P(0,3)', 'P(2,0)'}), frozenset({'P(1,0)', 'P(3,0)', 'P(0,2)', 'P(0,1)', 'P(0,0)', 'P(2,1)', 'P(0,3)', 'P(2,0)'}), frozenset({'P(2,3)', 'P(1,0)', 'P(3,0)', 'P(0,2)', 'P(0,1)', 'P(0,0)', 'P(0,3)', 'P(2,0)'}), frozenset({'P(1,0)', 'P(3,0)', 'P(0,2)', 'P(0,1)', 'P(0,0)', 'P(3,1)', 'P(0,3)', 'P(2,0)'})} 0.1156481110814518


In [69]:
# Evaluated using multiprocessing in the file "calculate_measure.py" where the results are saved to a .txt file 
with open("data_trine_2023_06_14-14_47_51.txt", "r") as f:
    all_measures = eval(f.read())

Now we implement the generalised measure to check consistency with the above and find minimum measure values

In [4]:
def search_all_combinations_tensor(group):
    """Find the summation of all possible combinations of groups"""
    max_len = int(len(group) / 2)
    dim = list(tetra_tensor.values())[0].rows
    # Add empty set edge case
    sub_groups = {frozenset(): sp.zeros(dim), frozenset(group.keys()): sp.eye(dim)}

    for i in range(max_len):
        combs = combinations(group, i + 1)
        for selection in combs:
            duplicate_group = dict(group)
            for prob in selection:
                duplicate_group.pop(prob)
            list1 = [group.get(key) for key in selection]
            list2 = list(duplicate_group.values())

            sub_groups[frozenset(selection)] = sum(list1, sp.zeros(dim))
            sub_groups[frozenset(duplicate_group.keys())] = sum(list2, sp.zeros(dim))

    return sub_groups

def measure_all_probabilities_mathematica_tensor(group_tensor):
    total_measures = len(group_tensor)
    print(f"Calculating {total_measures} measures...")
    
    measures = {}
    start = time.time()
    with WolframLanguageSession('C:\\Program Files\\Wolfram Research\\Wolfram Engine\\13.2\\WolframKernel.exe') as session:
        for i, (key, item) in enumerate(group_tensor.items()):
            if i%100 == 0 and i != 0:
                duration = time.time() - start
                calc_per_sec = i/duration
                time_remaining = (total_measures - i)/(calc_per_sec)
                hours = time_remaining//3600
                minutes = (time_remaining%3600)//60
                print(f"Calculated {i} measures, {calc_per_sec} it/s, {hours}H{minutes}M remaining")
                
            command = 'N[Eigenvalues[' + mathematica_code(item) + ']]'
            eigen_vals = session.evaluate(wlexpr(command))
            
            max_eig = max(eigen_vals)
            min_eig = min(eigen_vals)
            
            if max_eig <= 1 - 2*min_eig:
                (x, y) = (max_eig, (1-max_eig)/2)
            elif 1 - 2*min_eig <= max_eig and max_eig <= 1 - min_eig/2:
                (x, y) = (max_eig, min_eig)
            elif 1 - min_eig/2 <= max_eig:
                (x, y) = ((1-min_eig)/2, min_eig)
            else:
                raise ValueError(f"{key} failed, position {i}")
            
            measure = np.sqrt(1/3*((1-x-y)**2+(1-x)*y))
            measures[key] = measure
        
    return measures

In [5]:
phi0 = Qubit(1, 0)
phi1 = Qubit(1 / sqrt(3), sqrt(2 / 3))
phi2 = Qubit(1 / sqrt(3), sp.E ** (sp.I * 2 * sp.pi / 3) * sqrt(2 / 3))
phi3 = Qubit(1 / sqrt(3), sp.E ** (-sp.I * 2 * sp.pi / 3) * sqrt(2 / 3))

M0 = simp(1 / 2) * phi0.state * phi0.dag
M1 = simp(1 / 2) * phi1.state * phi1.dag
M2 = simp(1 / 2) * phi2.state * phi2.dag
M3 = simp(1 / 2) * phi3.state * phi3.dag

tetra = [M0, M1, M2, M3]

tetra_tensor = make_tensor(tetra)

groups = search_all_combinations_tensor(tetra_tensor)

In [74]:
all_measures = measure_all_probabilities_mathematica_tensor(groups)

Calculating 65536 measures...
Calculated 100 measures, 10.24918698486693 it/s, 1.0H46.0M remaining
Calculated 200 measures, 10.973633191636164 it/s, 1.0H39.0M remaining
Calculated 300 measures, 11.24856251966702 it/s, 1.0H36.0M remaining
Calculated 400 measures, 11.253116030682078 it/s, 1.0H36.0M remaining
Calculated 500 measures, 11.195421478162467 it/s, 1.0H36.0M remaining
Calculated 600 measures, 11.223263162458677 it/s, 1.0H36.0M remaining
Calculated 700 measures, 11.198554513972896 it/s, 1.0H36.0M remaining
Calculated 800 measures, 11.084535597732245 it/s, 1.0H37.0M remaining
Calculated 900 measures, 10.91998839416593 it/s, 1.0H38.0M remaining
Calculated 1000 measures, 10.909969025346935 it/s, 1.0H38.0M remaining
Calculated 1100 measures, 10.912402420648318 it/s, 1.0H38.0M remaining
Calculated 1200 measures, 10.901720269787017 it/s, 1.0H38.0M remaining
Calculated 1300 measures, 10.89345631744545 it/s, 1.0H38.0M remaining
Calculated 1400 measures, 10.876115979883593 it/s, 1.0H38.0M

Calculated 11700 measures, 9.901481390107033 it/s, 1.0H30.0M remaining
Calculated 11800 measures, 9.895638214851854 it/s, 1.0H30.0M remaining
Calculated 11900 measures, 9.8892247543217 it/s, 1.0H30.0M remaining
Calculated 12000 measures, 9.881568933249547 it/s, 1.0H30.0M remaining
Calculated 12100 measures, 9.878705416510826 it/s, 1.0H30.0M remaining
Calculated 12200 measures, 9.874913771554905 it/s, 1.0H30.0M remaining
Calculated 12300 measures, 9.874179129945919 it/s, 1.0H29.0M remaining
Calculated 12400 measures, 9.871491320913323 it/s, 1.0H29.0M remaining
Calculated 12500 measures, 9.869879321120631 it/s, 1.0H29.0M remaining
Calculated 12600 measures, 9.863877799423387 it/s, 1.0H29.0M remaining
Calculated 12700 measures, 9.85921895004441 it/s, 1.0H29.0M remaining
Calculated 12800 measures, 9.861324702576233 it/s, 1.0H29.0M remaining
Calculated 12900 measures, 9.861167708611333 it/s, 1.0H28.0M remaining
Calculated 13000 measures, 9.861622133919505 it/s, 1.0H28.0M remaining
Calculate

Calculated 23300 measures, 9.779649342633045 it/s, 1.0H11.0M remaining
Calculated 23400 measures, 9.780343996647593 it/s, 1.0H11.0M remaining
Calculated 23500 measures, 9.780200296514689 it/s, 1.0H11.0M remaining
Calculated 23600 measures, 9.779474128218377 it/s, 1.0H11.0M remaining
Calculated 23700 measures, 9.778191198910068 it/s, 1.0H11.0M remaining
Calculated 23800 measures, 9.777150136003288 it/s, 1.0H11.0M remaining
Calculated 23900 measures, 9.77854553429783 it/s, 1.0H10.0M remaining
Calculated 24000 measures, 9.779326878996832 it/s, 1.0H10.0M remaining
Calculated 24100 measures, 9.780238035116257 it/s, 1.0H10.0M remaining
Calculated 24200 measures, 9.781139851257194 it/s, 1.0H10.0M remaining
Calculated 24300 measures, 9.779706805821991 it/s, 1.0H10.0M remaining
Calculated 24400 measures, 9.779778558847964 it/s, 1.0H10.0M remaining
Calculated 24500 measures, 9.778673702630384 it/s, 1.0H9.0M remaining
Calculated 24600 measures, 9.776164501339531 it/s, 1.0H9.0M remaining
Calculate

Calculated 35000 measures, 9.68276563755544 it/s, 0.0H52.0M remaining
Calculated 35100 measures, 9.682959875625224 it/s, 0.0H52.0M remaining
Calculated 35200 measures, 9.683422104556046 it/s, 0.0H52.0M remaining
Calculated 35300 measures, 9.68405037214301 it/s, 0.0H52.0M remaining
Calculated 35400 measures, 9.684580577003594 it/s, 0.0H51.0M remaining
Calculated 35500 measures, 9.68425640466408 it/s, 0.0H51.0M remaining
Calculated 35600 measures, 9.684715046366486 it/s, 0.0H51.0M remaining
Calculated 35700 measures, 9.684660082473547 it/s, 0.0H51.0M remaining
Calculated 35800 measures, 9.684320454081718 it/s, 0.0H51.0M remaining
Calculated 35900 measures, 9.684233753544188 it/s, 0.0H51.0M remaining
Calculated 36000 measures, 9.684055128218878 it/s, 0.0H50.0M remaining
Calculated 36100 measures, 9.684142061871619 it/s, 0.0H50.0M remaining
Calculated 36200 measures, 9.683523950901423 it/s, 0.0H50.0M remaining
Calculated 36300 measures, 9.68322516094538 it/s, 0.0H50.0M remaining
Calculated

Calculated 46600 measures, 9.668946282844919 it/s, 0.0H32.0M remaining
Calculated 46700 measures, 9.66833738095199 it/s, 0.0H32.0M remaining
Calculated 46800 measures, 9.668072652486606 it/s, 0.0H32.0M remaining
Calculated 46900 measures, 9.667993021565014 it/s, 0.0H32.0M remaining
Calculated 47000 measures, 9.66754927071521 it/s, 0.0H31.0M remaining
Calculated 47100 measures, 9.667868845643467 it/s, 0.0H31.0M remaining
Calculated 47200 measures, 9.667786772276537 it/s, 0.0H31.0M remaining
Calculated 47300 measures, 9.667967139226986 it/s, 0.0H31.0M remaining
Calculated 47400 measures, 9.667950248002935 it/s, 0.0H31.0M remaining
Calculated 47500 measures, 9.667889848134793 it/s, 0.0H31.0M remaining
Calculated 47600 measures, 9.667903256840953 it/s, 0.0H30.0M remaining
Calculated 47700 measures, 9.668041082201759 it/s, 0.0H30.0M remaining
Calculated 47800 measures, 9.667591190721224 it/s, 0.0H30.0M remaining
Calculated 47900 measures, 9.667086555902516 it/s, 0.0H30.0M remaining
Calculat

Calculated 58200 measures, 9.59472199934718 it/s, 0.0H12.0M remaining
Calculated 58300 measures, 9.594106980244748 it/s, 0.0H12.0M remaining
Calculated 58400 measures, 9.593642363267458 it/s, 0.0H12.0M remaining
Calculated 58500 measures, 9.592220668670807 it/s, 0.0H12.0M remaining
Calculated 58600 measures, 9.591697813869944 it/s, 0.0H12.0M remaining
Calculated 58700 measures, 9.591835301199058 it/s, 0.0H11.0M remaining
Calculated 58800 measures, 9.592123101211335 it/s, 0.0H11.0M remaining
Calculated 58900 measures, 9.591977141059893 it/s, 0.0H11.0M remaining
Calculated 59000 measures, 9.591433951222282 it/s, 0.0H11.0M remaining
Calculated 59100 measures, 9.590901051725133 it/s, 0.0H11.0M remaining
Calculated 59200 measures, 9.590079030062395 it/s, 0.0H11.0M remaining
Calculated 59300 measures, 9.5898596832285 it/s, 0.0H10.0M remaining
Calculated 59400 measures, 9.589691404398868 it/s, 0.0H10.0M remaining
Calculated 59500 measures, 9.58908098030306 it/s, 0.0H10.0M remaining
Calculated

In [7]:
with open(f"two_tetra_povm_measurements_eigen_analysis_{datetime.datetime.now().strftime('%Y_%m_%d-%H_%M_%S')}.txt", "w") as f:
    f.write(str(all_measures))

In [9]:
# Check if the protocol which was found to be lowest using the simple CNOT unitary is in this new minimised group
group, measure = lowest_measure(all_measures)
print(len(group), measure)
frozenset({'(0,0)', '(0,1)', '(0,2)', '(0,3)', '(1,0)', '(2,0)', '(3,0)', '(1,1)'}) in group

144 0.07804803195715068


False

In [8]:
with open("two_tetra_povm_measurements_eigen_analysis_2023_06_22-12_36_47.txt", "r") as f:
    all_measures = eval(f.read())

In [212]:
group, measure = lowest_measure(all_measures)

In [216]:
# check all protocols are of length 8
truth = []
for g in group:
    truth.append(len(g) == 8)

all(truth)

True

In [4]:
# Check these eigenvalues more rigorously 
keys = frozenset({'(0,2)', '(0,3)', '(3,1)', '(0,1)', '(0,0)', '(3,3)', '(1,3)', '(1,1)'})

eigen_vals_sympy = groups[keys].eigenvals()

with WolframLanguageSession('C:\\Program Files\\Wolfram Research\\Wolfram Engine\\13.2\\WolframKernel.exe') as session:
    command = 'Eigenvalues[' + mathematica_code(groups[keys]) + ']'
    eigen_vals_mathematica = session.evaluate(wlexpr(command))

NameError: name 'groups' is not defined

In [40]:
# insert these into mathemtica notebook for eaiser reading
for e in eigen_vals_mathematica:
    print(e)

Times[Rational[1, 12], Plus[6, Power[Times[2, Plus[5, Power[21, Rational[1, 2]]]], Rational[1, 2]]]]
Times[Rational[1, 12], Plus[6, Power[Times[2, Plus[5, Times[-1, Power[21, Rational[1, 2]]]]], Rational[1, 2]]]]
Times[Rational[1, 12], Plus[6, Times[-1, Power[Times[2, Plus[5, Times[-1, Power[21, Rational[1, 2]]]]], Rational[1, 2]]]]]
Times[Rational[1, 12], Plus[6, Times[-1, Power[Times[2, Plus[5, Power[21, Rational[1, 2]]]], Rational[1, 2]]]]]


In [41]:
list(e_vals.keys())[0]

-sqrt(2)*sqrt(5 - sqrt(21))/12 + 1/2

In [42]:
list(e_vals.keys())[1]

sqrt(2)*sqrt(5 - sqrt(21))/12 + 1/2

In [43]:
list(e_vals.keys())[2]

-sqrt(2)*sqrt(sqrt(21) + 5)/12 + 1/2

In [44]:
list(e_vals.keys())[3]

sqrt(2)*sqrt(sqrt(21) + 5)/12 + 1/2

In [55]:
x = max(list(e_vals.keys()))
y = min(list(e_vals.keys()))

sp.simplify((1/3*((1-x-y)**2 + (1-x)*y))**0.5).evalf()

0.0780480319571507

Check the eigen value method gives the same results as the integration method

In [4]:
def simplified_root_mean_square_measure(probability, a, b):
    x = probability.subs({a: 1, b: 0}).evalf()
    y = probability.subs({a: 0, b: 1}).evalf()
    
    p_plus = probability.subs({a: 1/sqrt(2), b: 1/sqrt(2)}).evalf()
    p_minus = probability.subs({a: 1/sqrt(2), b: -1/sqrt(2)}).evalf()
    p_iplus = probability.subs({a: 1/sqrt(2), b: sp.I/sqrt(2)}).evalf()
    p_iminus = probability.subs({a: 1/sqrt(2), b: -sp.I/sqrt(2)}).evalf()
    z = 0.5*((p_plus - p_minus) - sp.I*(p_iplus - p_iminus)).evalf()
    
    measure = sqrt(1/3*((1-x-y)**2 + (1-x)*y + z*sp.conjugate(z)))
    
    return measure

In [5]:
phi0 = Qubit(1, 0)
phi1 = Qubit(1 / sqrt(3), sqrt(2 / 3))
phi2 = Qubit(1 / sqrt(3), sp.E ** (sp.I * 2 * sp.pi / 3) * sqrt(2 / 3))
phi3 = Qubit(1 / sqrt(3), sp.E ** (-sp.I * 2 * sp.pi / 3) * sqrt(2 / 3))

M0 = simp(1 / 2) * phi0.state * phi0.dag
M1 = simp(1 / 2) * phi1.state * phi1.dag
M2 = simp(1 / 2) * phi2.state * phi2.dag
M3 = simp(1 / 2) * phi3.state * phi3.dag

tetra = [M0, M1, M2, M3]

tetra_tensor = make_tensor(tetra)

a = sp.Symbol("a")
b = sp.Symbol("b")

probabilities = {}
for key, tet in tetra_tensor.items():
    probabilities[key] = calc_probabilities(tet, a, b)

groups = search_all_combinations(probabilities)

In [191]:
# Check result for the best case we know about
key = frozenset({'P(0,0)', 'P(0,1)', 'P(0,2)', 'P(0,3)', 'P(1,0)', 'P(2,0)', 'P(3,0)', 'P(1,1)'})
simplified_root_mean_square_measure(groups[key], a, b).evalf()

0.115648111081452

In [172]:
# Compare to previous result calculated using the integral. They look identitcal up to a rounding error
with open("two_tetra_povm_measurements_2023_06_18-09_02_21.txt", "r") as f:
    all_measures = eval(f.read())
    
all_measures[key]

0.1156481110814518

In [187]:
all_measures = measure_all_probabilities(groups, a, b, simplified_root_mean_square_measure)

Calculating 65536 measures...
Calculated 100 measures, 27.808695083714085 it/s, 0.0H39.0M remaining
Calculated 200 measures, 26.268073061159498 it/s, 0.0H41.0M remaining
Calculated 300 measures, 25.36652444876915 it/s, 0.0H42.0M remaining
Calculated 400 measures, 25.27003319463888 it/s, 0.0H42.0M remaining
Calculated 500 measures, 24.83656368825965 it/s, 0.0H43.0M remaining
Calculated 600 measures, 24.770760345115427 it/s, 0.0H43.0M remaining
Calculated 700 measures, 24.75256229213123 it/s, 0.0H43.0M remaining
Calculated 800 measures, 24.726666776515696 it/s, 0.0H43.0M remaining
Calculated 900 measures, 24.76530447179738 it/s, 0.0H43.0M remaining
Calculated 1000 measures, 24.728564569541003 it/s, 0.0H43.0M remaining
Calculated 1100 measures, 24.618881484247783 it/s, 0.0H43.0M remaining
Calculated 1200 measures, 24.431253887005116 it/s, 0.0H43.0M remaining
Calculated 1300 measures, 23.983348962741015 it/s, 0.0H44.0M remaining
Calculated 1400 measures, 23.882542264716623 it/s, 0.0H44.0M 

Calculated 11700 measures, 22.05289699835529 it/s, 0.0H40.0M remaining
Calculated 11800 measures, 22.051628635731735 it/s, 0.0H40.0M remaining
Calculated 11900 measures, 22.046926938247505 it/s, 0.0H40.0M remaining
Calculated 12000 measures, 22.03976222309012 it/s, 0.0H40.0M remaining
Calculated 12100 measures, 22.0390509275423 it/s, 0.0H40.0M remaining
Calculated 12200 measures, 22.035763430056534 it/s, 0.0H40.0M remaining
Calculated 12300 measures, 22.02985349629267 it/s, 0.0H40.0M remaining
Calculated 12400 measures, 22.03637965568196 it/s, 0.0H40.0M remaining
Calculated 12500 measures, 22.0323998280003 it/s, 0.0H40.0M remaining
Calculated 12600 measures, 22.027727891813356 it/s, 0.0H40.0M remaining
Calculated 12700 measures, 22.021051607869378 it/s, 0.0H39.0M remaining
Calculated 12800 measures, 22.019340978301116 it/s, 0.0H39.0M remaining
Calculated 12900 measures, 22.00221969538504 it/s, 0.0H39.0M remaining
Calculated 13000 measures, 21.993635793871157 it/s, 0.0H39.0M remaining
C

Calculated 23200 measures, 21.95339143108735 it/s, 0.0H32.0M remaining
Calculated 23300 measures, 21.950951561502706 it/s, 0.0H32.0M remaining
Calculated 23400 measures, 21.94679398033035 it/s, 0.0H31.0M remaining
Calculated 23500 measures, 21.942745467350818 it/s, 0.0H31.0M remaining
Calculated 23600 measures, 21.93866597276832 it/s, 0.0H31.0M remaining
Calculated 23700 measures, 21.93490730678339 it/s, 0.0H31.0M remaining
Calculated 23800 measures, 21.934951342914683 it/s, 0.0H31.0M remaining
Calculated 23900 measures, 21.939438300671732 it/s, 0.0H31.0M remaining
Calculated 24000 measures, 21.938974633192 it/s, 0.0H31.0M remaining
Calculated 24100 measures, 21.94338133767678 it/s, 0.0H31.0M remaining
Calculated 24200 measures, 21.941235212238386 it/s, 0.0H31.0M remaining
Calculated 24300 measures, 21.94422569476019 it/s, 0.0H31.0M remaining
Calculated 24400 measures, 21.94376406182151 it/s, 0.0H31.0M remaining
Calculated 24500 measures, 21.942759587064295 it/s, 0.0H31.0M remaining
Ca

Calculated 34700 measures, 21.87590680438123 it/s, 0.0H23.0M remaining
Calculated 34800 measures, 21.87552247434041 it/s, 0.0H23.0M remaining
Calculated 34900 measures, 21.876256087970123 it/s, 0.0H23.0M remaining
Calculated 35000 measures, 21.875564950673027 it/s, 0.0H23.0M remaining
Calculated 35100 measures, 21.875217824323833 it/s, 0.0H23.0M remaining
Calculated 35200 measures, 21.873892955908286 it/s, 0.0H23.0M remaining
Calculated 35300 measures, 21.87317766646624 it/s, 0.0H23.0M remaining
Calculated 35400 measures, 21.873440630525028 it/s, 0.0H22.0M remaining
Calculated 35500 measures, 21.872603836508784 it/s, 0.0H22.0M remaining
Calculated 35600 measures, 21.871355690072082 it/s, 0.0H22.0M remaining
Calculated 35700 measures, 21.870304374900893 it/s, 0.0H22.0M remaining
Calculated 35800 measures, 21.872203374636612 it/s, 0.0H22.0M remaining
Calculated 35900 measures, 21.87313103394173 it/s, 0.0H22.0M remaining
Calculated 36000 measures, 21.87440391611531 it/s, 0.0H22.0M remaini

Calculated 46200 measures, 21.804505542189393 it/s, 0.0H14.0M remaining
Calculated 46300 measures, 21.802557767645503 it/s, 0.0H14.0M remaining
Calculated 46400 measures, 21.803512855473183 it/s, 0.0H14.0M remaining
Calculated 46500 measures, 21.802519200364497 it/s, 0.0H14.0M remaining
Calculated 46600 measures, 21.79952728372781 it/s, 0.0H14.0M remaining
Calculated 46700 measures, 21.7970173505534 it/s, 0.0H14.0M remaining
Calculated 46800 measures, 21.796734532942118 it/s, 0.0H14.0M remaining
Calculated 46900 measures, 21.796245899214217 it/s, 0.0H14.0M remaining
Calculated 47000 measures, 21.795344317533615 it/s, 0.0H14.0M remaining
Calculated 47100 measures, 21.79401129017179 it/s, 0.0H14.0M remaining
Calculated 47200 measures, 21.79137704552774 it/s, 0.0H14.0M remaining
Calculated 47300 measures, 21.789854924530502 it/s, 0.0H13.0M remaining
Calculated 47400 measures, 21.790628201627296 it/s, 0.0H13.0M remaining
Calculated 47500 measures, 21.790064245040032 it/s, 0.0H13.0M remaini

Calculated 57800 measures, 21.73479761577045 it/s, 0.0H5.0M remaining
Calculated 57900 measures, 21.734308239172464 it/s, 0.0H5.0M remaining
Calculated 58000 measures, 21.73285545708533 it/s, 0.0H5.0M remaining
Calculated 58100 measures, 21.73171932726626 it/s, 0.0H5.0M remaining
Calculated 58200 measures, 21.731968228571137 it/s, 0.0H5.0M remaining
Calculated 58300 measures, 21.731701543887453 it/s, 0.0H5.0M remaining
Calculated 58400 measures, 21.7316047240449 it/s, 0.0H5.0M remaining
Calculated 58500 measures, 21.73084125226619 it/s, 0.0H5.0M remaining
Calculated 58600 measures, 21.729988882885287 it/s, 0.0H5.0M remaining
Calculated 58700 measures, 21.73024331555426 it/s, 0.0H5.0M remaining
Calculated 58800 measures, 21.730590481877794 it/s, 0.0H5.0M remaining
Calculated 58900 measures, 21.73310231728481 it/s, 0.0H5.0M remaining
Calculated 59000 measures, 21.73279959055305 it/s, 0.0H5.0M remaining
Calculated 59100 measures, 21.73232654149413 it/s, 0.0H4.0M remaining
Calculated 59200

In [201]:
with open(f"two_tetra_povm_measurements_simplified_{datetime.datetime.now().strftime('%Y_%m_%d-%H_%M_%S')}.txt", "w") as f:
    f.write(str(all_measures))

In [6]:
with open("two_tetra_povm_measurements_simplified_2023_06_22-14_52_49.txt", "r") as f:
    all_measures_simplified = eval(f.read())
with open("two_tetra_povm_measurements_2023_06_18-09_02_21.txt", "r") as f:
    all_measures = eval(f.read())

In [210]:
consistence_list = []
for key, item in all_measures_simplified.items():
    consistence_list.append(np.isclose(item, all_measures[key])) 
    
print(len(consistence_list))
print(all(consistence_list))

65536
True


Double checking code gives expected results with trine measurement

In [7]:
q0 = Qubit(1, 0)
q1 = Qubit(0, 1)

phi1 = Qubit(1/2, sqrt(3)/2)
phi2 = Qubit(1/2, -sqrt(3)/2)

In [8]:
M0 = simp(2/3) * q0.state*q0.dag
M1 = simp(2/3) * phi1.state*phi1.dag
M2 = simp(2/3) * phi2.state*phi2.dag

trine = [M0, M1, M2]

trine_tensor = make_tensor(trine)

groups = search_all_combinations_tensor(trine_tensor)

In [12]:
all_measures = measure_all_probabilities_mathematica_tensor(groups)

Calculating 512 measures...


WolframKernelException: Failed to communicate with kernel: C:\Program Files\Wolfram Research\Wolfram Engine\13.2\WolframKernel.exe.

Investigating symmetries in degenerate ideal protocols

In [10]:
with open("two_tetra_povm_measurements_eigen_analysis_2023_06_22-12_36_47.txt", "r") as f:
    all_measures = eval(f.read())

In [11]:
# Check if the protocol which was found to be lowest using the simple CNOT unitary is in this new minimised group
group, measure = lowest_measure(all_measures)
print(len(group), measure)
frozenset({'(0,0)', '(0,1)', '(0,2)', '(0,3)', '(1,0)', '(2,0)', '(3,0)', '(1,1)'}) in group

144 0.07804803195715068


False

In [13]:
with WolframLanguageSession('C:\\Program Files\\Wolfram Research\\Wolfram Engine\\13.2\\WolframKernel.exe') as session:          
    command = '2+2'
    x = session.evaluate(wlexpr(command))
    
print(x)

WolframKernelException: Failed to communicate with kernel: C:\Program Files\Wolfram Research\Wolfram Engine\13.2\WolframKernel.exe.